In [0]:
from pyspark.sql.functions import to_timestamp,hour,minute,when,col,current_timestamp,date_format,lit,unix_timestamp,expr,abs
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.tuning import TrainValidationSplit
import datetime

In [0]:
p1_weather_df = spark.table('solarPowerdata.plant1_weather_sensor_data')

In [0]:
display(p1_weather_df.summary())

summary,DATE_TIME,PLANT_ID,SOURCE_KEY,AMBIENT_TEMPERATURE,MODULE_TEMPERATURE,IRRADIATION
count,3182,3182,3182,3182,3182,3182
mean,null,4135001.0,null,25.5316057175644,31.091015378426455,0.228312830065651
stddev,null,0.0,null,3.354855664435953,12.261221623078292,0.3008363668152484
min,2020-05-15 00:00:00,4135001,HmiyD2TTLFNqkNe,20.398504866666666,18.140415466666663,0.0
25%,null,4135001,null,22.704483933333336,21.08964233333333,0.0
50%,null,4135001,null,24.6126954,24.617095866666666,0.024557107333333335
75%,null,4135001,null,27.920831266666664,41.316923333333335,0.4497933986666667
max,2020-06-17 23:45:00,4135001,HmiyD2TTLFNqkNe,35.25248613333334,65.54571366666664,1.2216518466666668


In [0]:
p1_gen_df = spark.table('SolarPowerData.plant1_generation_data')

In [0]:
display(p1_gen_df.summary())

summary,DATE_TIME,PLANT_ID,SOURCE_KEY,DC_POWER,AC_POWER,DAILY_YIELD,TOTAL_YIELD
count,68778,68778,68778,68778,68778,68778,68778
mean,null,4135001.0,null,3147.4262112269307,307.802752265517,3295.9687371825785,6978711.7606710475
stddev,null,0.0,null,4036.457168799247,394.3964386953456,3145.1783088786433,416271.98285620863
min,01-06-2020 00:00,4135001,1BY6WEcLGh8j5v7,0.0,0.0,0.0,6183645.0
25%,null,4135001,null,0.0,0.0,0.0,6512002.0
50%,null,4135001,null,427.625,41.3375,2658.0,7146685.0
75%,null,4135001,null,6365.375,623.4625,6274.0,7268656.571
max,31-05-2020 23:45,4135001,zVJPv84UY57bAof,14471.125,1410.95,9163.0,7846821.0


In [0]:
display(p1_gen_df.groupBy('SOURCE_KEY').count())

SOURCE_KEY,count
1BY6WEcLGh8j5v7,3154
adLQvlD726eNBSB,3119
rGa61gmuvPhdLxV,3124
YxYtjZvoooNbGkE,3104
pkci93gMrogZuBj,3125
bvBOhCH3iADSZry,3155
sjndEbLyjtCKgGv,3124
zVJPv84UY57bAof,3124
VHMLBKoKgIrUVDU,3133
uHbuxQJl8lW7ozc,3125


In [0]:
display(p1_gen_df)

DATE_TIME,PLANT_ID,SOURCE_KEY,DC_POWER,AC_POWER,DAILY_YIELD,TOTAL_YIELD
15-05-2020 00:00,4135001,1BY6WEcLGh8j5v7,0.0,0.0,0.0,6259559.0
15-05-2020 00:00,4135001,1IF53ai7Xc0U56Y,0.0,0.0,0.0,6183645.0
15-05-2020 00:00,4135001,3PZuoBAID5Wc2HD,0.0,0.0,0.0,6987759.0
15-05-2020 00:00,4135001,7JYdWkrLSPkdwr4,0.0,0.0,0.0,7602960.0
15-05-2020 00:00,4135001,McdE0feGgRqW7Ca,0.0,0.0,0.0,7158964.0
15-05-2020 00:00,4135001,VHMLBKoKgIrUVDU,0.0,0.0,0.0,7206408.0
15-05-2020 00:00,4135001,WRmjgnKYAwPKWDb,0.0,0.0,0.0,7028673.0
15-05-2020 00:00,4135001,ZnxXDlPa8U1GXgE,0.0,0.0,0.0,6522172.0
15-05-2020 00:00,4135001,ZoEaEvLYb1n2sOq,0.0,0.0,0.0,7098099.0
15-05-2020 00:00,4135001,adLQvlD726eNBSB,0.0,0.0,0.0,6271355.0


In [0]:
p1_gen_df = p1_gen_df.withColumn('DATE_TIME',to_timestamp(p1_gen_df.DATE_TIME,'dd-MM-yyyy HH:mm'))
p1_weather_df = p1_weather_df.withColumn('DATE_TIME',to_timestamp(p1_weather_df.DATE_TIME,'yyyy-MM-dd HH:mm:ss'))

In [0]:
display(p1_weather_df)

DATE_TIME,PLANT_ID,SOURCE_KEY,AMBIENT_TEMPERATURE,MODULE_TEMPERATURE,IRRADIATION
2020-05-15T00:00:00.000+0000,4135001,HmiyD2TTLFNqkNe,25.184316133333333,22.8575074,0.0
2020-05-15T00:15:00.000+0000,4135001,HmiyD2TTLFNqkNe,25.08458866666667,22.761667866666663,0.0
2020-05-15T00:30:00.000+0000,4135001,HmiyD2TTLFNqkNe,24.935752600000004,22.592305533333334,0.0
2020-05-15T00:45:00.000+0000,4135001,HmiyD2TTLFNqkNe,24.8461304,22.360852133333335,0.0
2020-05-15T01:00:00.000+0000,4135001,HmiyD2TTLFNqkNe,24.621525357142858,22.165422642857145,0.0
2020-05-15T01:15:00.000+0000,4135001,HmiyD2TTLFNqkNe,24.536092200000002,21.968570866666667,0.0
2020-05-15T01:30:00.000+0000,4135001,HmiyD2TTLFNqkNe,24.638673866666664,22.352925666666668,0.0
2020-05-15T01:45:00.000+0000,4135001,HmiyD2TTLFNqkNe,24.873022333333335,23.1609192,0.0
2020-05-15T02:00:00.000+0000,4135001,HmiyD2TTLFNqkNe,24.936930466666663,23.026113,0.0
2020-05-15T02:15:00.000+0000,4135001,HmiyD2TTLFNqkNe,25.012247600000002,23.343229266666665,0.0


In [0]:
p1_raw_df = p1_weather_df.select('DATE_TIME','AMBIENT_TEMPERATURE','MODULE_TEMPERATURE','IRRADIATION').join(p1_gen_df, on = 'DATE_TIME',how = 'inner')
p1_raw_df = p1_raw_df.withColumn('TIME',date_format('DATE_TIME','HH:mm:ss'))

p1_df = p1_raw_df.filter(col('TIME').between('06:00:00','18:15:00'))
p1_plantoff_df = p1_raw_df.filter((col('TIME')>=lit('18:30:00')) | (col('TIME')<=lit('05:45:00')))

In [0]:
display(p1_df)

DATE_TIME,AMBIENT_TEMPERATURE,MODULE_TEMPERATURE,IRRADIATION,PLANT_ID,SOURCE_KEY,DC_POWER,AC_POWER,DAILY_YIELD,TOTAL_YIELD,TIME
2020-05-15T06:00:00.000+0000,24.088446066666666,22.206756600000002,0.0058869571866666665,4135001,1BY6WEcLGh8j5v7,37.14285714,3.585714286,0.0,6259559.0,06:00:00
2020-05-15T06:00:00.000+0000,24.088446066666666,22.206756600000002,0.0058869571866666665,4135001,1IF53ai7Xc0U56Y,53.5,5.1625,0.0,6183645.0,06:00:00
2020-05-15T06:00:00.000+0000,24.088446066666666,22.206756600000002,0.0058869571866666665,4135001,3PZuoBAID5Wc2HD,58.0,5.585714286,0.0,6987759.0,06:00:00
2020-05-15T06:00:00.000+0000,24.088446066666666,22.206756600000002,0.0058869571866666665,4135001,7JYdWkrLSPkdwr4,58.42857143,5.628571429,0.0,7602960.0,06:00:00
2020-05-15T06:00:00.000+0000,24.088446066666666,22.206756600000002,0.0058869571866666665,4135001,McdE0feGgRqW7Ca,54.375,5.25,0.0,7158964.0,06:00:00
2020-05-15T06:00:00.000+0000,24.088446066666666,22.206756600000002,0.0058869571866666665,4135001,VHMLBKoKgIrUVDU,0.0,0.0,0.0,7206408.0,06:00:00
2020-05-15T06:00:00.000+0000,24.088446066666666,22.206756600000002,0.0058869571866666665,4135001,WRmjgnKYAwPKWDb,43.85714286,4.228571429,0.0,7028673.0,06:00:00
2020-05-15T06:00:00.000+0000,24.088446066666666,22.206756600000002,0.0058869571866666665,4135001,ZnxXDlPa8U1GXgE,58.57142857,5.671428571,0.0,6522172.0,06:00:00
2020-05-15T06:00:00.000+0000,24.088446066666666,22.206756600000002,0.0058869571866666665,4135001,ZoEaEvLYb1n2sOq,24.5,2.366666667,0.0,7098099.0,06:00:00
2020-05-15T06:00:00.000+0000,24.088446066666666,22.206756600000002,0.0058869571866666665,4135001,adLQvlD726eNBSB,56.125,5.425,0.0,6271355.0,06:00:00


In [0]:
p1_train_df ,p1_test_df= p1_df.randomSplit([0.8,0.2])

In [0]:
display(p1_df)

DATE_TIME,AMBIENT_TEMPERATURE,MODULE_TEMPERATURE,IRRADIATION,PLANT_ID,SOURCE_KEY,DC_POWER,AC_POWER,DAILY_YIELD,TOTAL_YIELD,TIME
2020-05-15T06:00:00.000+0000,24.088446066666666,22.206756600000002,0.0058869571866666665,4135001,1BY6WEcLGh8j5v7,37.14285714,3.585714286,0.0,6259559.0,06:00:00
2020-05-15T06:00:00.000+0000,24.088446066666666,22.206756600000002,0.0058869571866666665,4135001,1IF53ai7Xc0U56Y,53.5,5.1625,0.0,6183645.0,06:00:00
2020-05-15T06:00:00.000+0000,24.088446066666666,22.206756600000002,0.0058869571866666665,4135001,3PZuoBAID5Wc2HD,58.0,5.585714286,0.0,6987759.0,06:00:00
2020-05-15T06:00:00.000+0000,24.088446066666666,22.206756600000002,0.0058869571866666665,4135001,7JYdWkrLSPkdwr4,58.42857143,5.628571429,0.0,7602960.0,06:00:00
2020-05-15T06:00:00.000+0000,24.088446066666666,22.206756600000002,0.0058869571866666665,4135001,McdE0feGgRqW7Ca,54.375,5.25,0.0,7158964.0,06:00:00
2020-05-15T06:00:00.000+0000,24.088446066666666,22.206756600000002,0.0058869571866666665,4135001,VHMLBKoKgIrUVDU,0.0,0.0,0.0,7206408.0,06:00:00
2020-05-15T06:00:00.000+0000,24.088446066666666,22.206756600000002,0.0058869571866666665,4135001,WRmjgnKYAwPKWDb,43.85714286,4.228571429,0.0,7028673.0,06:00:00
2020-05-15T06:00:00.000+0000,24.088446066666666,22.206756600000002,0.0058869571866666665,4135001,ZnxXDlPa8U1GXgE,58.57142857,5.671428571,0.0,6522172.0,06:00:00
2020-05-15T06:00:00.000+0000,24.088446066666666,22.206756600000002,0.0058869571866666665,4135001,ZoEaEvLYb1n2sOq,24.5,2.366666667,0.0,7098099.0,06:00:00
2020-05-15T06:00:00.000+0000,24.088446066666666,22.206756600000002,0.0058869571866666665,4135001,adLQvlD726eNBSB,56.125,5.425,0.0,6271355.0,06:00:00


In [0]:
display(p1_df.filter(col('DC_POWER')==0))

DATE_TIME,AMBIENT_TEMPERATURE,MODULE_TEMPERATURE,IRRADIATION,PLANT_ID,SOURCE_KEY,DC_POWER,AC_POWER,DAILY_YIELD,TOTAL_YIELD,TIME
2020-05-15T06:00:00.000+0000,24.088446066666666,22.206756600000002,0.0058869571866666665,4135001,VHMLBKoKgIrUVDU,0.0,0.0,0.0,7206408.0,06:00:00
2020-05-15T09:15:00.000+0000,27.988362071428575,46.617706500000004,0.3448840357142857,4135001,zVJPv84UY57bAof,0.0,0.0,711.0,7116862.0,09:15:00
2020-05-15T09:30:00.000+0000,27.516727866666667,39.136326,0.2492484,4135001,zVJPv84UY57bAof,0.0,0.0,711.0,7116862.0,09:30:00
2020-05-17T06:00:00.000+0000,22.693113133333334,21.416339066666666,0.00478462602,4135001,1BY6WEcLGh8j5v7,0.0,0.0,0.0,6271605.0,06:00:00
2020-05-17T06:00:00.000+0000,22.693113133333334,21.416339066666666,0.00478462602,4135001,1IF53ai7Xc0U56Y,0.0,0.0,0.0,6196594.0,06:00:00
2020-05-17T06:00:00.000+0000,22.693113133333334,21.416339066666666,0.00478462602,4135001,3PZuoBAID5Wc2HD,0.0,0.0,0.0,7000689.0,06:00:00
2020-05-17T06:00:00.000+0000,22.693113133333334,21.416339066666666,0.00478462602,4135001,7JYdWkrLSPkdwr4,0.0,0.0,0.0,7615537.0,06:00:00
2020-05-17T06:00:00.000+0000,22.693113133333334,21.416339066666666,0.00478462602,4135001,McdE0feGgRqW7Ca,0.0,0.0,0.0,7172021.0,06:00:00
2020-05-17T06:00:00.000+0000,22.693113133333334,21.416339066666666,0.00478462602,4135001,VHMLBKoKgIrUVDU,0.0,0.0,0.0,7219380.0,06:00:00
2020-05-17T06:00:00.000+0000,22.693113133333334,21.416339066666666,0.00478462602,4135001,WRmjgnKYAwPKWDb,0.0,0.0,0.0,7041430.0,06:00:00


In [0]:
display(p1_df)

DATE_TIME,AMBIENT_TEMPERATURE,MODULE_TEMPERATURE,IRRADIATION,PLANT_ID,SOURCE_KEY,DC_POWER,AC_POWER,DAILY_YIELD,TOTAL_YIELD,TIME
2020-05-15T06:00:00.000+0000,24.088446066666666,22.206756600000002,0.0058869571866666665,4135001,1BY6WEcLGh8j5v7,37.14285714,3.585714286,0.0,6259559.0,06:00:00
2020-05-15T06:00:00.000+0000,24.088446066666666,22.206756600000002,0.0058869571866666665,4135001,1IF53ai7Xc0U56Y,53.5,5.1625,0.0,6183645.0,06:00:00
2020-05-15T06:00:00.000+0000,24.088446066666666,22.206756600000002,0.0058869571866666665,4135001,3PZuoBAID5Wc2HD,58.0,5.585714286,0.0,6987759.0,06:00:00
2020-05-15T06:00:00.000+0000,24.088446066666666,22.206756600000002,0.0058869571866666665,4135001,7JYdWkrLSPkdwr4,58.42857143,5.628571429,0.0,7602960.0,06:00:00
2020-05-15T06:00:00.000+0000,24.088446066666666,22.206756600000002,0.0058869571866666665,4135001,McdE0feGgRqW7Ca,54.375,5.25,0.0,7158964.0,06:00:00
2020-05-15T06:00:00.000+0000,24.088446066666666,22.206756600000002,0.0058869571866666665,4135001,VHMLBKoKgIrUVDU,0.0,0.0,0.0,7206408.0,06:00:00
2020-05-15T06:00:00.000+0000,24.088446066666666,22.206756600000002,0.0058869571866666665,4135001,WRmjgnKYAwPKWDb,43.85714286,4.228571429,0.0,7028673.0,06:00:00
2020-05-15T06:00:00.000+0000,24.088446066666666,22.206756600000002,0.0058869571866666665,4135001,ZnxXDlPa8U1GXgE,58.57142857,5.671428571,0.0,6522172.0,06:00:00
2020-05-15T06:00:00.000+0000,24.088446066666666,22.206756600000002,0.0058869571866666665,4135001,ZoEaEvLYb1n2sOq,24.5,2.366666667,0.0,7098099.0,06:00:00
2020-05-15T06:00:00.000+0000,24.088446066666666,22.206756600000002,0.0058869571866666665,4135001,adLQvlD726eNBSB,56.125,5.425,0.0,6271355.0,06:00:00


In [0]:
va = VectorAssembler().setInputCols(['IRRADIATION','AMBIENT_TEMPERATURE']).setOutputCol('features')
p1_train_df = va.transform(p1_train_df)

In [0]:
p1_train_df = p1_train_df.withColumnRenamed('AC_POWER','label')
display(p1_train_df)

DATE_TIME,AMBIENT_TEMPERATURE,MODULE_TEMPERATURE,IRRADIATION,PLANT_ID,SOURCE_KEY,DC_POWER,label,DAILY_YIELD,TOTAL_YIELD,TIME,features
2020-05-15T06:00:00.000+0000,24.088446066666666,22.206756600000002,0.0058869571866666665,4135001,1IF53ai7Xc0U56Y,53.5,5.1625,0.0,6183645.0,06:00:00,"Map(vectorType -> dense, length -> 2, values -> List(0.0058869571866666665, 24.088446066666666))"
2020-05-15T06:00:00.000+0000,24.088446066666666,22.206756600000002,0.0058869571866666665,4135001,7JYdWkrLSPkdwr4,58.42857143,5.628571429,0.0,7602960.0,06:00:00,"Map(vectorType -> dense, length -> 2, values -> List(0.0058869571866666665, 24.088446066666666))"
2020-05-15T06:00:00.000+0000,24.088446066666666,22.206756600000002,0.0058869571866666665,4135001,VHMLBKoKgIrUVDU,0.0,0.0,0.0,7206408.0,06:00:00,"Map(vectorType -> dense, length -> 2, values -> List(0.0058869571866666665, 24.088446066666666))"
2020-05-15T06:00:00.000+0000,24.088446066666666,22.206756600000002,0.0058869571866666665,4135001,WRmjgnKYAwPKWDb,43.85714286,4.228571429,0.0,7028673.0,06:00:00,"Map(vectorType -> dense, length -> 2, values -> List(0.0058869571866666665, 24.088446066666666))"
2020-05-15T06:00:00.000+0000,24.088446066666666,22.206756600000002,0.0058869571866666665,4135001,ZnxXDlPa8U1GXgE,58.57142857,5.671428571,0.0,6522172.0,06:00:00,"Map(vectorType -> dense, length -> 2, values -> List(0.0058869571866666665, 24.088446066666666))"
2020-05-15T06:00:00.000+0000,24.088446066666666,22.206756600000002,0.0058869571866666665,4135001,ZoEaEvLYb1n2sOq,24.5,2.366666667,0.0,7098099.0,06:00:00,"Map(vectorType -> dense, length -> 2, values -> List(0.0058869571866666665, 24.088446066666666))"
2020-05-15T06:00:00.000+0000,24.088446066666666,22.206756600000002,0.0058869571866666665,4135001,adLQvlD726eNBSB,56.125,5.425,0.0,6271355.0,06:00:00,"Map(vectorType -> dense, length -> 2, values -> List(0.0058869571866666665, 24.088446066666666))"
2020-05-15T06:00:00.000+0000,24.088446066666666,22.206756600000002,0.0058869571866666665,4135001,bvBOhCH3iADSZry,37.0,3.571428571,0.0,6316803.0,06:00:00,"Map(vectorType -> dense, length -> 2, values -> List(0.0058869571866666665, 24.088446066666666))"
2020-05-15T06:00:00.000+0000,24.088446066666666,22.206756600000002,0.0058869571866666665,4135001,iCRJl6heRkivqQ3,41.85714286,4.028571429,0.0,7177992.0,06:00:00,"Map(vectorType -> dense, length -> 2, values -> List(0.0058869571866666665, 24.088446066666666))"
2020-05-15T06:00:00.000+0000,24.088446066666666,22.206756600000002,0.0058869571866666665,4135001,ih0vzX44oOqAx2f,50.16666667,4.85,0.0,6185184.0,06:00:00,"Map(vectorType -> dense, length -> 2, values -> List(0.0058869571866666665, 24.088446066666666))"


In [0]:
lr  = LinearRegression().setFeaturesCol('features').setLabelCol('label')

In [0]:
lr_model = lr.fit(p1_train_df)

In [0]:
lr_model.summary.meanAbsoluteError

Out[22]: 44.91843870324977

In [0]:
lr_model.setPredictionCol('ACPower_Predicted')

Out[23]: LinearRegressionModel: uid=LinearRegression_4e3866a87046, numFeatures=2

In [0]:
lr_model.write().overwrite().save('LR_FaultDetectionModel')

In [0]:
pred_p1_train_df = lr_model.transform(p1_train_df)
pred_p1_train_df = pred_p1_train_df.withColumn('residuals',col('ACPower_Predicted') - col('label') )
train_fault_instances_df =  pred_p1_train_df.filter(col('residuals')>=pred_p1_train_df.stat.approxQuantile('residuals',[0.995],0.00)[0])

In [0]:
display(pred_p1_train_df)

DATE_TIME,AMBIENT_TEMPERATURE,MODULE_TEMPERATURE,IRRADIATION,PLANT_ID,SOURCE_KEY,DC_POWER,label,DAILY_YIELD,TOTAL_YIELD,TIME,features,ACPower_Predicted,residuals
2020-05-15T06:00:00.000+0000,24.088446066666666,22.206756600000002,0.0058869571866666665,4135001,1IF53ai7Xc0U56Y,53.5,5.1625,0.0,6183645.0,06:00:00,"Map(vectorType -> dense, length -> 2, values -> List(0.0058869571866666665, 24.088446066666666))",36.44899920078846,31.286499200788455
2020-05-15T06:00:00.000+0000,24.088446066666666,22.206756600000002,0.0058869571866666665,4135001,7JYdWkrLSPkdwr4,58.42857143,5.628571429,0.0,7602960.0,06:00:00,"Map(vectorType -> dense, length -> 2, values -> List(0.0058869571866666665, 24.088446066666666))",36.44899920078846,30.820427771788456
2020-05-15T06:00:00.000+0000,24.088446066666666,22.206756600000002,0.0058869571866666665,4135001,VHMLBKoKgIrUVDU,0.0,0.0,0.0,7206408.0,06:00:00,"Map(vectorType -> dense, length -> 2, values -> List(0.0058869571866666665, 24.088446066666666))",36.44899920078846,36.44899920078846
2020-05-15T06:00:00.000+0000,24.088446066666666,22.206756600000002,0.0058869571866666665,4135001,WRmjgnKYAwPKWDb,43.85714286,4.228571429,0.0,7028673.0,06:00:00,"Map(vectorType -> dense, length -> 2, values -> List(0.0058869571866666665, 24.088446066666666))",36.44899920078846,32.22042777178846
2020-05-15T06:00:00.000+0000,24.088446066666666,22.206756600000002,0.0058869571866666665,4135001,ZnxXDlPa8U1GXgE,58.57142857,5.671428571,0.0,6522172.0,06:00:00,"Map(vectorType -> dense, length -> 2, values -> List(0.0058869571866666665, 24.088446066666666))",36.44899920078846,30.777570629788457
2020-05-15T06:00:00.000+0000,24.088446066666666,22.206756600000002,0.0058869571866666665,4135001,ZoEaEvLYb1n2sOq,24.5,2.366666667,0.0,7098099.0,06:00:00,"Map(vectorType -> dense, length -> 2, values -> List(0.0058869571866666665, 24.088446066666666))",36.44899920078846,34.08233253378846
2020-05-15T06:00:00.000+0000,24.088446066666666,22.206756600000002,0.0058869571866666665,4135001,adLQvlD726eNBSB,56.125,5.425,0.0,6271355.0,06:00:00,"Map(vectorType -> dense, length -> 2, values -> List(0.0058869571866666665, 24.088446066666666))",36.44899920078846,31.023999200788456
2020-05-15T06:00:00.000+0000,24.088446066666666,22.206756600000002,0.0058869571866666665,4135001,bvBOhCH3iADSZry,37.0,3.571428571,0.0,6316803.0,06:00:00,"Map(vectorType -> dense, length -> 2, values -> List(0.0058869571866666665, 24.088446066666666))",36.44899920078846,32.87757062978846
2020-05-15T06:00:00.000+0000,24.088446066666666,22.206756600000002,0.0058869571866666665,4135001,iCRJl6heRkivqQ3,41.85714286,4.028571429,0.0,7177992.0,06:00:00,"Map(vectorType -> dense, length -> 2, values -> List(0.0058869571866666665, 24.088446066666666))",36.44899920078846,32.420427771788454
2020-05-15T06:00:00.000+0000,24.088446066666666,22.206756600000002,0.0058869571866666665,4135001,ih0vzX44oOqAx2f,50.16666667,4.85,0.0,6185184.0,06:00:00,"Map(vectorType -> dense, length -> 2, values -> List(0.0058869571866666665, 24.088446066666666))",36.44899920078846,31.598999200788455


In [0]:
outlier_residual_limit = pred_p1_train_df.stat.approxQuantile('residuals',[0.95],0.05)

In [0]:
display(p1_test_df.filter(col('DC_POWER')!=0))

DATE_TIME,AMBIENT_TEMPERATURE,MODULE_TEMPERATURE,IRRADIATION,PLANT_ID,SOURCE_KEY,DC_POWER,AC_POWER,DAILY_YIELD,TOTAL_YIELD,TIME
2020-05-15T06:00:00.000+0000,24.088446066666666,22.206756600000002,0.0058869571866666665,4135001,1BY6WEcLGh8j5v7,37.14285714,3.585714286,0.0,6259559.0,06:00:00
2020-05-15T06:00:00.000+0000,24.088446066666666,22.206756600000002,0.0058869571866666665,4135001,3PZuoBAID5Wc2HD,58.0,5.585714286,0.0,6987759.0,06:00:00
2020-05-15T06:00:00.000+0000,24.088446066666666,22.206756600000002,0.0058869571866666665,4135001,McdE0feGgRqW7Ca,54.375,5.25,0.0,7158964.0,06:00:00
2020-05-15T06:15:00.000+0000,24.01163526666667,22.35345866666667,0.02228160746666667,4135001,3PZuoBAID5Wc2HD,318.625,30.775,3.5,6987762.5,06:15:00
2020-05-15T06:15:00.000+0000,24.01163526666667,22.35345866666667,0.02228160746666667,4135001,WRmjgnKYAwPKWDb,315.0,30.4375,3.375,7028676.375,06:15:00
2020-05-15T06:15:00.000+0000,24.01163526666667,22.35345866666667,0.02228160746666667,4135001,adLQvlD726eNBSB,315.1428571,30.44285714,3.428571429,6271358.429,06:15:00
2020-05-15T06:15:00.000+0000,24.01163526666667,22.35345866666667,0.02228160746666667,4135001,uHbuxQJl8lW7ozc,311.5714286,30.08571429,3.142857143,7038684.143,06:15:00
2020-05-15T06:15:00.000+0000,24.01163526666667,22.35345866666667,0.02228160746666667,4135001,z9Y9gH1T5YWrNuG,312.7142857,30.18571429,3.428571429,7007869.429,06:15:00
2020-05-15T06:30:00.000+0000,23.976731266666665,22.893282,0.049409723866666665,4135001,VHMLBKoKgIrUVDU,716.1428571,69.42857143,7.142857143,7206415.143,06:30:00
2020-05-15T06:30:00.000+0000,23.976731266666665,22.893282,0.049409723866666665,4135001,iCRJl6heRkivqQ3,677.0,65.5875,14.5,7178006.5,06:30:00


In [0]:
p1_test_df = va.transform(p1_test_df)
p1_test_df = p1_test_df.withColumnRenamed('AC_POWER','label')


In [0]:

pred_p1_test_df = lr_model.transform(p1_test_df)
pred_p1_test_df = pred_p1_test_df.withColumn('residuals',col('ACPower_Predicted')-col('label'))
fault_instances_df=  pred_p1_test_df.filter(col('residuals')>=pred_p1_test_df.stat.approxQuantile('residuals',[0.995],0.05)[0])

In [0]:
pred_p1_df = pred_p1_train_df.union(pred_p1_test_df)

In [0]:
p1_plantoff_df = va.transform(p1_plantoff_df).withColumnRenamed('AC_POWER','label').withColumn('ACPower_Predicted',lit(0)).withColumn('residuals',lit(0)).withColumn('Fault_Flag',lit(0))

In [0]:
pred_p1_df = pred_p1_df.withColumn('Fault_Flag',expr(f"case when abs(residuals) > {pred_p1_df.stat.approxQuantile(('residuals'),[0.995],0.00)[0]} then 1 else 0 end "))
p1_rul_raw_df = pred_p1_df.union(p1_plantoff_df)
p1_rul_raw_df=  p1_rul_raw_df.select('DATE_TIME','AMBIENT_TEMPERATURE','MODULE_TEMPERATURE','IRRADIATION','PLANT_ID','SOURCE_KEY','DC_POWER','label','DAILY_YIELD','TOTAL_YIELD','Fault_Flag')
p1_rul_raw_df = p1_rul_raw_df.withColumnRenamed('label','AC_POWER')
p1_rul_raw_df.write.mode('overwrite').option('overwriteSchema','true').saveAsTable('SolarPowerData.RUL_DATASET')